In [1]:
import numpy as np
from skimage import io, color, exposure, transform
from skimage.color import rgb2gray
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split  #it came from update scikit learn. https://stackoverflow.com/questions/40704484/importerror-no-module-named-model-selection
import os
import glob
import h5py

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D,Conv1D
from keras.layers.pooling import MaxPooling2D

from keras.layers.convolutional import Convolution2D, MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_first')

from matplotlib import pyplot as plt
#%matplotlib inline

#import keras

NUM_CLASSES = 5
IMG_SIZE = 48
IMG_depth = 3

Using TensorFlow backend.


In [2]:
def preprocess_img(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,centre[1]-min_side//2:centre[1]+min_side//2,:]
#    img = rgb2gray(img)

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1)

    return img

def get_class(img_path):
    return int(img_path.split('/')[-2])

In [3]:
imgs = []
labels = []
root_dir = '/media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/train_image_AI/'
#path='/home/atif/training_by_several_learning_process/flower_photos/00000/'

#all_img_paths = glob.glob(path+ '5547758_eea9edfd54_n_000.jpg')

all_img_paths = glob.glob(os.path.join(root_dir, '*/*.png')) #I have done the training with .ppm format image. If another type of image will come 
                                                                                    #them .ppm will be changed by that extension
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    try:
        img = preprocess_img(io.imread(img_path))
        label = get_class(img_path)
        imgs.append(img)
        labels.append(label)

        if len(imgs)%1000 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
            #print("get it 2")
    except (IOError, OSError):
        print('missed', img_path)
        pass

X = np.array(imgs, dtype='float32') #Keeping the image as an array
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/color/colorconv.py:283: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]
/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]


Processed 1000/5400
Processed 2000/5400
Processed 3000/5400
Processed 4000/5400
Processed 5000/5400


In [4]:
X = X.reshape(len(imgs),IMG_depth,IMG_SIZE,IMG_SIZE) # write (IMG_SIZE,IMG_SIZE,1 if you want channel last; 1= grayscale;3=RGB)
# plt.imshow(X[0],cmap="gray")
print(X.shape)
print(X.ndim)
print(X[0].shape)

print(X.shape)
print(Y.shape)

(5400, 3, 48, 48)
4
(3, 48, 48)
(5400, 3, 48, 48)
(5400, 5)


In [5]:
def cnn_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(3, IMG_SIZE, IMG_SIZE),
                     activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

model = cnn_model()

lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 48, 48)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 46, 46)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 23, 23)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 23, 23)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 23, 23)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 21, 21)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 10, 10)        0         
__________

In [7]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 32
epochs = 50
model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          #np.resize(img, (-1, <image shape>)
          callbacks=[LearningRateScheduler(lr_schedule),ModelCheckpoint('/media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/trained_file_AI/trained_UE4_first.h5', save_best_only=True)])

Train on 4320 samples, validate on 1080 samples
Epoch 1/50
4320/4320 [==============================] - 149s 34ms/step - loss: 1.5412 - acc: 0.2933 - val_loss: 1.2160 - val_acc: 0.5269
Epoch 2/50
4320/4320 [==============================] - 184s 43ms/step - loss: 0.7465 - acc: 0.7299 - val_loss: 0.1564 - val_acc: 0.9620
Epoch 3/50
4320/4320 [==============================] - 207s 48ms/step - loss: 0.2080 - acc: 0.9294 - val_loss: 0.0398 - val_acc: 0.9870
Epoch 4/50
4320/4320 [==============================] - 183s 42ms/step - loss: 0.0723 - acc: 0.9759 - val_loss: 0.0227 - val_acc: 0.9954
Epoch 5/50
4320/4320 [==============================] - 220s 51ms/step - loss: 0.0524 - acc: 0.9843 - val_loss: 0.0117 - val_acc: 0.9944
Epoch 6/50
4320/4320 [==============================] - 193s 45ms/step - loss: 0.0291 - acc: 0.9912 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 7/50
4320/4320 [==============================] - 198s 46ms/step - loss: 0.0316 - acc: 0.9910 - val_loss: 0.0023 - val_acc: 

In [10]:
from keras.models import load_model
model = load_model('/media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/trained_file_AI/trained_UE4_first.h5')
#for gray scale
def preprocess_img(img):
#     Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]
#     img = rgb2gray(img)

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1)

    return img

import glob

path = r'/media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/'

img_path = glob.glob(path+ '/*.png')
for image in img_path:
    X_test=[]
    X_test.append(preprocess_img(io.imread(image)))
    X_test = np.array(X_test)
#     plt.imshow(X_test)
    X_test = X_test.reshape(len(X_test),3,IMG_SIZE,IMG_SIZE)
    
    print("\n",image)
    predicted_class = model.predict_classes(X_test)
    print("predicted class: ",predicted_class)
    
    probability = model.predict_proba(X_test)
    print("probability: ",probability)

/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/color/colorconv.py:283: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]
/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "



 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/calgonit_finish_1035.png
predicted class:  [4]
probability:  [[5.0177057e-10 1.0891918e-07 1.1687316e-13 1.2897646e-14 9.9999988e-01]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/calgonit_finish_24.png
predicted class:  [4]
probability:  [[3.1424308e-15 1.0334130e-16 2.2029010e-24 4.2919256e-19 1.0000000e+00]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/calgonit_finish_435.png
predicted class:  [4]
probability:  [[8.3640532e-07 2.5469612e-07 4.1909775e-08 3.0858408e-07 9.9999857e-01]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/calgonit_finish_824.png
predicted class:  [4]
probability:  [[2.7730483e-08 1.1543107e-06 1.0534397e-09 3.1265267e-07 9.9999845e-01]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/calgonit_finish_974.png
predicted class

/home/atif/anaconda3/envs/venv/lib/python3.6/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]



 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/rock_44.png
predicted class:  [1]
probability:  [[2.1815778e-05 9.9990058e-01 4.1056985e-08 7.1170070e-05 6.4108613e-06]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/rock_875.png
predicted class:  [1]
probability:  [[1.0950030e-08 1.0000000e+00 1.1585048e-09 3.7393374e-08 2.9445759e-09]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/somat_1004.png
predicted class:  [3]
probability:  [[3.0130477e-13 1.6989387e-09 3.7450887e-09 1.0000000e+00 1.5859566e-18]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/somat_34.png
predicted class:  [3]
probability:  [[3.663658e-11 5.127386e-11 9.343606e-10 1.000000e+00 2.028886e-08]]

 /media/atif/0820209220208930/unreal_cv_documentation/ignore_from_git/test_image_AI/somat_374.png
predicted class:  [3]
probability:  [[2.4770513e-10 9.1767366e-10 3.49914